In [1]:
# importing required packages
import numpy as np 
import imutils
import cv2
from imutils.perspective import four_point_transform
from imutils import contours


In [2]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}          # correct answers 

image = cv2.imread('test_01.png')                    # load bubble sheet
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)       # covert to gray
blurred= cv2.GaussianBlur(gray, (5,5), 0)            # blur image
edged= cv2.Canny(blurred, 75, 200)                   # edge detection


# find lis of contours in edge detected image
# to identify our bubble sheet
cnts= cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts= imutils.grab_contours(cnts)
docCnt= None

# proced if one or more contour is detected
if len(cnts)>0:

    # sort contours acc to area in reverse order
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # loop over each contour 
    # approx each contour 
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx= cv2.approxPolyDP(c, peri*0.02, True)
        
        # if four points are detected then we got our bubble sheet
        # rectangle contour 
        if len(approx)==4:
            docCnt = approx
            break

# get four point trasform   
paper= four_point_transform(image, docCnt.reshape(4,2))
warped= four_point_transform(gray, docCnt.reshape(4,2))

# convert image to binary
thresh= cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# uncomment and execute to see binary image 
#cv2.imshow("thresh", thresh)

# find each bubble in transformed image
cnts= cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts= imutils.grab_contours(cnts)
questionCnts = []

for c in cnts:
    (x, y, w, h)= cv2.boundingRect(c)
    ar= w/float(h)
    # removing any extra detected contour 
    if w>=20 and h>=20 and ar>=0.9 and ar<=1.1:
        questionCnts.append(c)

# sorting list of contours from top to bottom
questionCnts = contours.sort_contours(questionCnts, method='top-to-bottom')[0]

correct= 0

# loop over every five contours 
# sort those five contours in left to right direction
for (q, i) in enumerate( np.arange(0, len(questionCnts), 5)):

    cnts= contours.sort_contours(questionCnts[i:i+5])[0]
    
    bubbled= None
    
    # loop over left to right sorted contours
    for (j, c) in enumerate(cnts):
        mask= np.zeros(thresh.shape, dtype= "uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        mask= cv2.bitwise_and(thresh, thresh, mask=mask)
        total= cv2.countNonZero(mask)

        if bubbled is None or total > bubbled[0]:
            bubbled= (total ,j)
    
    color= (0,0,255)
    k = ANSWER_KEY[q]
    
    # calculating correct answer 

    if k == bubbled[1]:
        color= (0,255,0)
        correct +=1

    cv2.drawContours(paper, [cnts[k]], -1, color, 3)

score = (correct/ 5.0)* 100

# showing final score and detected bubbles 

print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)
                                    

[INFO] score: 80.00%


-1